# extract_info.ipynb

Information of the song samples is obtained from the [.mp3 samples](../data/emotifymusic/) and saved in [info.csv](.../data/Info/).

In [1]:
import sys, os
import numpy as np
import pandas as pd
import librosa

In [2]:
working_dir = os.getcwd()

while working_dir[-11:] != "Informatics":
    os.chdir("..")
    working_dir = os.getcwd()

In [3]:
path = "data/emotifymusic/"
genres = ["classical", "electronic", "pop", "rock"]

In [6]:
import sys
import os
import numpy as np

sys.path.append(os.getcwd())

def extractInfoFromSounds(path, genre=None, print_message = True):
    """
    Extract info from sound files in folder
    Sound is converted to mono by averaging the two channels
    """
    
    genre_shift = 0
    
    if genre=="electronic":
        genre_shift = 100
    elif genre=="pop":
        genre_shift = 200
    elif genre=="rock":
        genre_shift = 300
    
    track_id_ = []
    names_ = []
    duration_ = []
    samplerate_ = []
    maxabsamplitude_ = []
    meanabsamplitude_ = []
    
    for file in os.scandir(path):
        if file.path.endswith(".mp3"):
            if print_message: print("analyzing ", file.name, file.path)

            currentpath = path + "/" + file.name
            names_.append(file.name)
            
            int_track = int(file.name[:-4])
            track_id_.append(int_track + genre_shift)
            
            sound, sample_rate = librosa.load(currentpath, sr=None, mono=True) 
            
            samplerate_.append(sample_rate)
            
            duration = librosa.get_duration(y=sound, sr=sample_rate)
            
            if duration < 59.0:
                print(int_track + genre_shift, duration)
            duration_.append(duration)
     
            maxabsamplitude_.append(np.max(np.abs(sound)))
            meanabsamplitude_.append(np.mean(np.abs(sound)))
            
            
    genre_ = [genre for i in range(len(duration_))]
    
    return pd.DataFrame({'Track ID' : track_id_, 'Genre' : genre_, 'Filename': names_, 'Duration' : duration_, 'Sample rate' : samplerate_ ,
                      'Max absolute amplitude' : maxabsamplitude_ ,
                       'Mean absolute amplitude' : meanabsamplitude_ })
    

            # dataframe append file.name, sample_rate, duration, max_abs_amp, mean_abs_amp

In [7]:
genres = ["classical", "electronic", "pop", "rock"]

df = pd.DataFrame({'Track ID' : [], 'Genre' : [], 'Filename': [], 'Duration' : [], 'Sample rate' : [] ,
                      'Max absolute amplitude' : [] ,
                       'Mean absolute amplitude' : []})

for genre in genres:
    sub_df = extractInfoFromSounds(path + genre, genre=genre, print_message=False)
    df = df.append(sub_df)

16 42.34448979591837
40 34.963356009070296
57 58.41569160997732
66 25.835102040816327
73 30.27591836734694


In [ ]:
pd.options.display.max_rows=1000
df

In [ ]:
df["Track ID"] = df["Track ID"].astype(np.int32)
df

In [ ]:
df.to_csv("Info/info.csv", index=False)